## 資料集合

In [1]:
x_set = []
y_set = []

## churn.all

In [2]:
import pandas as pd

#放在不同資料夾用絕對路徑
data = pd.read_csv('./data/churn.all.csv')

#刪除不要的欄位
data.drop(['id'], inplace=True, axis=1)
data.drop(['phone_number'], inplace=True, axis=1)

#將目標屬性轉為數字
import numpy as np
data['Churn'] = np.where(data['Churn'] == ' False', 0, 1)

#分割輸入屬性及目標屬性

#輸入屬性
x_ = data.drop(['Churn'], axis=1)#同上方法刪除行

#目標屬性
y_ = data['Churn']

#去除錯誤值及空值填補
from sklearn.pipeline import Pipeline
from data_preprocessing_pipeline import *
from feature_engineering import *

pipe = Pipeline(
       [
        ('define_column_type', define_column_type()),
        ('scaler', scaler()),
        ('dummies', dummies()),
        #('filter', feature_selection(method = 'anova_kf', p_threshold = 1)),
        #('sfs',feature_selection(method = 'sfs', sfs_direction = direction, sfs_scoring = scoring, sfs_cv = cv, sfs_estimator = estimator))
       ])

x_ = pipe.fit_transform(x_, y_)
print(x_.shape)

import collections
c = collections.Counter(y_)
print(c)

2022 16:45:44 INFO Fit Columns Type Already Success!!!
2022 16:45:44 INFO Transform Columns Type Success!!!
2022 16:45:44 INFO scaler Success!!!
2022 16:45:44 INFO dummies Success!!!


(5000, 87)
Counter({0: 4293, 1: 707})


In [8]:
x_set.append(x_)
y_set.append(y_)

## bank

In [7]:
import pandas as pd

#放在不同資料夾用絕對路徑
data = pd.read_csv('./data/bank.csv')

#刪除ID
#data.drop(['id'], inplace=True, axis=1)

#將目標屬性轉為數字
import numpy as np
data['y'] = np.where(data['y'] == 'no', 0, 1)

#分割輸入屬性及目標屬性

#輸入屬性
x_ = data.drop(['y'], axis=1)#同上方法刪除行
#x_ = data.drop(['phone_number'], axis=1)#同上方法刪除行

#目標屬性
y_ = data['y']

#去除錯誤值及空值填補
from sklearn.pipeline import Pipeline
from data_preprocessing_pipeline import *
from feature_engineering import *

pipe = Pipeline(
       [
#         ('Wrong_Value',wrong_value_fillna(wrong_value=['unknown'])),
#         ('Null_Value', xgb_fill()),
        ('define_column_type', define_column_type()),
        ('scaler', scaler()),
        ('dummies', dummies()),
       ])

x_ = pipe.fit_transform(x_, y_)
print(x_.shape)

import collections
c = collections.Counter(y_)
print(c)

2022 16:46:42 INFO Fit Columns Type Already Success!!!
2022 16:46:42 INFO Transform Columns Type Success!!!
2022 16:46:42 INFO scaler Success!!!
2022 16:46:42 INFO dummies Success!!!


(41188, 91)
Counter({0: 36548, 1: 4640})


## WPBC(不要)

## 用產生資料測試

## 1.Annealing(不要)

## 2.Statlog (Australian Credit Approval)

## 6.default of credit card clients

## 7.glass(不要，多類)

## 8.heart（不要，多類）

## 9.hepatitis

## 10.horse-colic(不要，不能讀)

## 11.ionosphere

## 12.iris(不要，多類)

## 16.sonar

## 17.tic-tac-toe

## 19.wine(不要，多類)

## 20.zoo(不要，多類)

## 21.broadband

## 分類測試

In [13]:
#計算f1函數
def get_f1(model, x, y):
    
    '''
    model : 用甚麼方式建模
    x : 輸入屬性
    y : 目標屬性
    '''
    import numpy as np
    from sklearn.metrics import f1_score
    
    #建模
    #model.fit(x, y)

    #用訓練資料測試
    f1 = f1_score(y, model.predict(x), average='binary')

#     print('這次的f1 : ', f1,'\n')
    return f1

In [14]:
#計算AUC函數
def get_auc(model, x, y):
    
    '''
    model : 用甚麼方式建模
    x : 輸入屬性
    y : 目標屬性
    '''
    import numpy as np
    from sklearn import metrics
    
    #建模
    #model.fit(x, y)

    #用訓練資料測試
    fpr, tpr, thresholds = metrics.roc_curve(y, model.predict(x))
    auc = metrics.auc(fpr, tpr)

#     print('這次的AUC : ', auc,'\n')
    return auc

In [18]:
#交叉驗證，分層隨機抽樣
from sklearn.model_selection import StratifiedKFold
import random as rd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

dataset_num = 1
for x_, y_ in zip(x_set, y_set):
    
    ans_auc = [] #儲存每次交叉驗證的AUC
    ans_f1 = [] #儲存每次交叉驗證的f1

    rand = rd.randint(1, 10) #產生交叉驗證抽樣隨機種子
    skf = StratifiedKFold(n_splits=10, random_state = rand, shuffle=True)

    for train_index, test_index in skf.split(x_, y_):
        #print("TRAIN:", train_index, "TEST:", test_index,'\n')
        x_train, x_test = x_.iloc[train_index], x_.iloc[test_index]
        y_train, y_test = y_.iloc[train_index], y_.iloc[test_index]

        x_train = pd.DataFrame(x_train)
        y_train = pd.DataFrame(y_train)

        x_test = pd.DataFrame(x_test)
        y_test = pd.DataFrame(y_test) 

        clf = DecisionTreeClassifier(random_state=0)
    #     clf = AdaBoostClassifier(n_estimators=100, random_state=0, algorithm='SAMME')
#         clf = AdaBoostClassifier(base_estimator=GaussianNB(), n_estimators=100, random_state=0, algorithm='SAMME')


        clf.fit(x_train, y_train)

        ans_f1.append(get_f1(clf,x_test,y_test)) #取得用測試資料測試的f1
        ans_auc.append(get_auc(clf,x_test,y_test)) #取得用測試資料測試的AUC


    print('dataset', dataset_num, 'F1:',np.mean(ans_f1))
    print('dataset', dataset_num, 'AUC:',np.mean(ans_auc))
    dataset_num = dataset_num + 1

dataset 1 F1: 0.7233169581063599
dataset 1 AUC: 0.842269200392358
dataset 2 F1: 0.5147876376762246
dataset 2 AUC: 0.7298700546109916


##